## Referencce website 
code from https://blog.csdn.net/dgvv4/article/details/122023047

### **Import library**

In [1]:
import cv2

In [2]:
import mediapipe as mp

In [3]:
import time

In [4]:
import csv
import os

In [5]:
import sqlite3

In [6]:
from datetime import datetime

### **Create database**

In [ ]:
#conn = sqlite3.connect('hand_coordinates.db')

In [ ]:
# 创建一个表
#c = conn.cursor()
# 创建用于保存归一化坐标的表
#c.execute('''
#CREATE TABLE IF NOT EXISTS normalized_coordinates (
    #id INTEGER PRIMARY KEY AUTOINCREMENT,
    #timestamp TEXT,
    #x REAL,
    #y REAL
#);
#''')

# 创建用于保存像素坐标的表
#c.execute('''
#CREATE TABLE IF NOT EXISTS pixel_coordinates (
   # id INTEGER PRIMARY KEY AUTOINCREMENT,
   # timestamp TEXT,
   # x INTEGER,
   # y INTEGER
#);
#''')
#conn.commit()

### **Visualize hand landmarks and connections**

In [7]:
cap = cv2.VideoCapture(0)#capture video, 0 means built-in webcam

In [8]:
mpHands = mp.solutions.hands

In [9]:
hands = mpHands.Hands(static_image_mode=False, #静态追踪，低于0.5置信度会再一次跟踪
                      max_num_hands=2, # 最多有2只手
                      min_detection_confidence=0.5, # 最小检测置信度
                      min_tracking_confidence=0.5)  # 最小跟踪置信度

In [10]:
mpDraw = mp.solutions.drawing_utils

In [11]:
pTime = 0 #处理一张图像前的时间
cTime = 0 #一张图处理完的时间

In [12]:
# 存放坐标信息
lmList = []
captured_coordinates = []

In [13]:
#使用数据库保存得到的坐标
conn = sqlite3.connect('hand_coordinates.db')
# 创建一个表
db = conn.cursor()
# 创建用于保存归一化坐标的表
db.execute('''
CREATE TABLE IF NOT EXISTS normalized_coordinates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    x REAL,
    y REAL
);
''')

# 创建用于保存像素坐标的表
db.execute('''
CREATE TABLE IF NOT EXISTS pixel_coordinates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    x INTEGER,
    y INTEGER
);
''')
conn.commit()
# 初始化
cap = cv2.VideoCapture(0)


normalized_coords = []
pixel_coords = []
lmList = []  
pTime = 0


In [14]:
# 创建保存数据的目录
output_dir = "output_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

while True:
    # 视频读取和预处理
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    print(results.multi_hand_landmarks)

    if results.multi_hand_landmarks:
        for handlms in results.multi_hand_landmarks:
            for index, lm in enumerate(handlms.landmark):
                if index == 8:  # 如果你只关心索引为8的关键点
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    print(index, cx, cy)
                    lmList.append([index, cx, cy])
                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

                    # 插入数据到数据库
                    db.execute("INSERT INTO normalized_coordinates (timestamp, x, y) VALUES (?, ?, ?)", (current_time, lm.x, lm.y))

                    # 将像素坐标保存到数据库
                    db.execute("INSERT INTO pixel_coordinates (timestamp, x, y) VALUES (?, ?, ?)", (current_time, cx, cy))

                    conn.commit()

                    # 在图像上画圆
                    cv2.circle(img, (cx, cy), 12, (0, 220, 23), cv2.FILLED)

            # 其他代码（绘制关键点等）

    # FPS显示和其他后处理
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

    cv2.imshow('Image', img)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# 关闭数据库连接
conn.close()
# 释放摄像头资源
cap.release()
# 关闭所有 OpenCV 窗口
cv2.destroyAllWindows()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[landmark {
  x: 0.1274779587984085
  y: 1.0585529804229736
  z: 7.819250669172106e-08
}
landmark {
  x: 0.18878239393234253
  y: 1.0100239515304565
  z: -0.01953350193798542
}
landmark {
  x: 0.23076863586902618
  y: 0.9289842844009399
  z: -0.027159539982676506
}
landmark {
  x: 0.2443229854106903
  y: 0.8637604117393494
  z: -0.036854375153779984
}
landmark {
  x: 0.24568593502044678
  y: 0.8048107028007507
  z: -0.04670359566807747
}
landmark {
  x: 0.1986451894044876
  y: 0.8197091817855835
  z: -0.003953264094889164
}
landmark {
  x: 0.21976108849048615
  y: 0.7468690276145935
  z: -0.024334514513611794
}
landmark {
  x: 0.23818477988243103
  y: 0.7077029347419739
  z: -0.04212348908185959
}
landmark {
  x: 0.2557344436645508
  y: 0.6794708371162415
  z: -0.05451901629567146
}
landmark {
  x: 0.14632421731948853
  y: 0.8228622674942017
  z: -0.00979116

In [15]:
#娴熟数据
conn = sqlite3.connect('hand_coordinates.db')
db = conn.cursor()
# 查询数据库中的数据
db.execute("SELECT * FROM normalized_coordinates")

# 获取查询结果
rows = db.fetchall()

# 打印查询结果
for row in rows:
    print(row)


(1, '2023-10-07 21:49:53.487964', 0.2557344436645508, 0.6794708371162415)
(2, '2023-10-07 21:49:53.627812', 0.194623202085495, 0.8125550746917725)
(3, '2023-10-07 21:49:53.875704', 0.5023647546768188, 0.6571882963180542)
(4, '2023-10-07 21:49:54.015044', 0.4774928689002991, 0.6942633390426636)
(5, '2023-10-07 21:49:54.124416', 0.46921610832214355, 0.739716112613678)
(6, '2023-10-07 21:49:54.359386', 0.3064225912094116, 0.5389569997787476)
(7, '2023-10-07 21:49:54.484589', 0.30421268939971924, 0.5862026810646057)
(8, '2023-10-07 21:49:54.737593', 0.50883549451828, 0.6337623000144958)
(9, '2023-10-07 21:49:54.864661', 0.47921323776245117, 0.5548879504203796)
(10, '2023-10-07 21:49:55.285686', 0.31408271193504333, 0.3925495445728302)
(11, '2023-10-07 21:49:55.393594', 0.24623894691467285, 0.7844346761703491)
(12, '2023-10-07 21:49:55.798775', 0.57454514503479, 0.6859130263328552)
(13, '2023-10-07 21:49:56.467066', 0.26472869515419006, 0.6468172073364258)
(14, '2023-10-07 21:49:57.047988',

In [17]:
#查看数据库
import pandas as pd

conn = sqlite3.connect('koto_hand_coordinates.db')

# 将数据读入Pandas DataFrame
df = pd.read_sql_query("SELECT * from normalized_coordinates", conn)

# 显示数据
print(df)

        id                   timestamp         x         y
0        1  2023-10-07 21:49:53.487964  0.255734  0.679471
1        2  2023-10-07 21:49:53.627812  0.194623  0.812555
2        3  2023-10-07 21:49:53.875704  0.502365  0.657188
3        4  2023-10-07 21:49:54.015044  0.477493  0.694263
4        5  2023-10-07 21:49:54.124416  0.469216  0.739716
...    ...                         ...       ...       ...
1079  1080  2023-10-07 21:52:24.630805  0.172806  0.538748
1080  1081  2023-10-07 21:52:24.689013  0.182194  0.663437
1081  1082  2023-10-07 21:52:24.751807  0.180511  0.651462
1082  1083  2023-10-07 21:52:24.814804  0.238076  0.780875
1083  1084  2023-10-07 21:52:24.876805  0.239940  0.798002

[1084 rows x 4 columns]
